# Test interpolazione a celle quadrate (in metri)

apro tutti gli nc di ogni mese, li croppo e li salvo nella cartella di lavoro


In [1]:
# import libraries for working with .nc
import xarray as xr
import os   
import numpy as np




In [2]:
# origin path

path =  '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily/'


carico un file di esempio

In [3]:
year = 2022
month = 6
# load this file s_subset.to_netcdf(outp + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month), encoding=encoding)as a xarray dataset
ds = xr.open_dataset(path + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month))


  # save the subset to a NetCDF file with the updated encoding (nome file= outp + year + 'CHL_03-10_{}_.nc'.format(year))
 #       ds_subset.to_netcdf(outp + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month), encoding=encoding)
               
ds


<xarray.Dataset>
Dimensions:    (time: 30, latitude: 1248, longitude: 5280)
Coordinates:
  * time       (time) datetime64[ns] 2022-06-01 2022-06-02 ... 2022-06-30
  * latitude   (latitude) float64 84.99 84.98 84.97 84.96 ... 72.03 72.02 72.01
  * longitude  (longitude) float64 -4.995 -4.984 -4.974 ... 49.97 49.98 49.99
Data variables:
    CHL        (time, latitude, longitude) float32 ...

interpolo la longitudine

aggrego 5 celle

In [4]:
# Calculate the differences in latitudes and longitudes
delta_latitude = abs(ds.latitude.values[1] - ds.latitude.values[0])
delta_longitude = abs(ds.longitude.values[1] - ds.longitude.values[0])

print(f"Spatial resolution (latitude): {delta_latitude} degrees")
print(f"Spatial resolution (longitude): {delta_longitude} degrees")



Spatial resolution (latitude): 0.010416666666671404 degrees
Spatial resolution (longitude): 0.010416666666685614 degrees


In [5]:
# Calculate the average latitude of the dataset
avg_latitude = ds.latitude.mean().values

# Convert the average latitude from degrees to radians
avg_latitude_rad = np.deg2rad(avg_latitude)

# Calculate the spatial resolution in meters
resolution_latitude_m = delta_latitude * 111000
resolution_longitude_m = delta_longitude * 111000 * np.cos(avg_latitude_rad)

print(f"Spatial resolution (latitude): {resolution_latitude_m} meters")
print(f"Spatial resolution (longitude): {resolution_longitude_m} meters")

resolution_latitude_m / resolution_longitude_m

Spatial resolution (latitude): 1156.2500000005257 meters
Spatial resolution (longitude): 230.51917417030356 meters


5.015851736247798

In [6]:
np.arange(ds.longitude[0], ds.longitude[-1], step = 5 * delta_longitude)

5 * delta_longitude

0.05208333333342807

In [7]:
ds_coarsened = ds.interp(longitude=np.arange(ds.longitude[0], ds.longitude[-1], step = 5 * delta_longitude), method='nearest')


print(ds_coarsened)

<xarray.Dataset>
Dimensions:    (time: 30, latitude: 1248, longitude: 1056)
Coordinates:
  * time       (time) datetime64[ns] 2022-06-01 2022-06-02 ... 2022-06-30
  * latitude   (latitude) float64 84.99 84.98 84.97 84.96 ... 72.03 72.02 72.01
  * longitude  (longitude) float64 -4.995 -4.943 -4.891 ... 49.85 49.9 49.95
Data variables:
    CHL        (time, latitude, longitude) float32 nan nan nan ... 0.141 0.1427


ricontrollo la risoluzione

In [23]:
# Calculate the differences in latitudes and longitudes
delta_latitude = abs(ds_coarsened.latitude.values[1] - ds_coarsened.latitude.values[0])
delta_longitude = abs(ds_coarsened.longitude.values[1] - ds_coarsened.longitude.values[0])

print(f"Spatial resolution (latitude): {delta_latitude} degrees")
print(f"Spatial resolution (longitude): {delta_longitude} degrees")

0.0104 *5

Spatial resolution (latitude): 0.010416666666671404 degrees
Spatial resolution (longitude): 0.05208333333342807 degrees


0.052

In [9]:
# Calculate the average latitude of the dataset
avg_latitude = ds_coarsened.latitude.mean().values

# Convert the average latitude from degrees to radians
avg_latitude_rad = np.deg2rad(avg_latitude)

# Calculate the spatial resolution in meters
resolution_latitude_m = delta_latitude * 111000
resolution_longitude_m = delta_longitude * 111000 * np.cos(avg_latitude_rad)

print(f"Spatial resolution (latitude): {resolution_latitude_m} meters")
print(f"Spatial resolution (longitude): {resolution_longitude_m} meters")

resolution_latitude_m / resolution_longitude_m

Spatial resolution (latitude): 1156.2500000005257 meters
Spatial resolution (longitude): 1152.5958708515177 meters


1.0031703472495597

le celle sono quasi quadrate!

In [11]:
# creo la cartella path =  '/mnt/d/Dropbox/R_projects/SV_phenology/data/test_coarsened e salvo il file
# creo la cartella se non esiste già
os.makedirs('/mnt/d/Dropbox/R_projects/SV_phenology/data/test_coarsened', exist_ok=True)

ds_coarsened.to_netcdf('/mnt/d/Dropbox/R_projects/SV_phenology/data/test_coarsened' + '/CHL_03-10_{}_{}_coarsenedv2.nc'.format(year, month))


# Loop for interpolation

In [25]:
path =  '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily/'


month = 3
year = 2022

path + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month)



pathout = '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily_coarsened/'
pathout + '{}'.format(year) + '/CHL_03-10_{}_{}_coarse.nc'.format(year, month)


'/mnt/d/Dropbox/R_projects/SV_phenology/data/daily_coarsened/2022/CHL_03-10_2022_3_coarse.nc'

In [28]:
path =  '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily/'

pathout = '/mnt/d/Dropbox/R_projects/SV_phenology/data/daily_coarsened/'


# set consistent _FillValue and missing_value attributes for the 'CHL' variable
encoding = {'CHL': {'_FillValue': -999.0, 'missing_value': -999.0}}


# loop for each year within 1998 and 2022
for year in range(1998,2023):
    # loop for each month within march and october

    # create a folder for each year in path outh, if does not exist already
    os.makedirs(pathout + '{}'.format(year), exist_ok=True)
    
    for month in range(3, 11):
        # temp path concatenate folder for year and month (e.g. 1998/03), with month in two digits

        ds = xr.open_dataset(path + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month))
        
        # read the size of the cells
        delta_longitude = abs(ds.longitude.values[1] - ds.longitude.values[0])


        # coarsen the dataset
        ds_coarsened = ds.interp(longitude=np.arange(ds.longitude[0], ds.longitude[-1], step = 5 * delta_longitude), method='nearest')


        # save the subset to a NetCDF file with the updated encoding (nome file= outp + year + 'CHL_03-10_{}_.nc'.format(year))
        ds_coarsened.to_netcdf(pathout + '{}'.format(year) + '/CHL_03-10_{}_{}.nc'.format(year, month), encoding=encoding)
               

        # close the dataset
        ds.close()

        # close the subset
        ds_coarsened.close()

        # print the year and month
        print(year, month)
        




 

1998 3
1998 4
1998 5
1998 6
1998 7
1998 8
1998 9
1998 10
1999 3
1999 4
1999 5
1999 6
1999 7
1999 8
1999 9
1999 10
2000 3
2000 4
2000 5
2000 6
2000 7
2000 8
2000 9
2000 10
2001 3
2001 4
2001 5
2001 6
2001 7
2001 8
2001 9
2001 10
2002 3
2002 4
2002 5
2002 6
2002 7
2002 8
2002 9
2002 10
2003 3
2003 4
2003 5
2003 6
2003 7
2003 8
2003 9
2003 10
2004 3
2004 4
2004 5
2004 6
2004 7
2004 8
2004 9
2004 10
2005 3
2005 4
2005 5
2005 6
2005 7
2005 8
2005 9
2005 10
2006 3
2006 4
2006 5
2006 6
2006 7
2006 8
2006 9
2006 10
2007 3
2007 4
2007 5
2007 6
2007 7
2007 8
2007 9
2007 10
2008 3
2008 4
2008 5
2008 6
2008 7
2008 8
2008 9
2008 10
2009 3
2009 4
2009 5
2009 6
2009 7
2009 8
2009 9
2009 10
2010 3
2010 4
2010 5
2010 6
2010 7
2010 8
2010 9
2010 10
2011 3
2011 4
2011 5
2011 6
2011 7
2011 8
2011 9
2011 10
2012 3
2012 4
2012 5
2012 6
2012 7
2012 8
2012 9
2012 10
2013 3
2013 4
2013 5
2013 6
2013 7
2013 8
2013 9
2013 10
2014 3
2014 4
2014 5
2014 6
2014 7
2014 8
2014 9
2014 10
2015 3
2015 4
2015 5
2015 6
201